<a id="OptTop"></a>
# Optimizers

 * This notebook serves as a library of optimizer classes that may be supplied with an objective function and boundary constraints
 * provides a set of test functions to visualize optimizer performance

## [Optimizer Interface](#OptInterface)
 - interface for each optimization class to follow

## [Optimization Algorithm Visualizer](#AlgVis)
 - set of 2D example problems which can be:
 - solved
 - visualied
 - with any samplin based opt method
 
## [Baysian Optimization](#bayes)
 - surogate model method
 - very sample efficient
 
## [Simulated Annealing](#SA)
 - implemented from scratch
 - gradually lower temp to choose between best and current sample

## [DIRECT Algorithm](#DIRECT)
 - global method based on partitioning parameter space hypercube

## [Covariance Matrix Adaptation (CMA)](#CMA)
 - evolutionary strategy 
 
## [Notes](#Notes)




In [1]:
#check if the notebook is being used as a library, or being run as a notebook
asLibrary = lambda : '__file__' in globals()
if asLibrary():
    print("optimizers run as library")

<a id="OptInterface"></a>
# Optimizer Interface [ &#x21ea;](#OptTop)
to maitain a level of decoupling between elements of the software system, the optimizer will not be given access to the robot or the experiment explicitly, but will deal instead  only with function handles for the objective function and constraint functions.

In [2]:
#define optimization interface here
class Opt:
    """
    defines the interface that all types of optimizer should follow
    """
    def optimize(self,obj,bounds,constraints=None,x0=None):
        """
        the function which actually performs the optimization or sampling
        inputs:
            obj - a function handle from exp.objective
            bounds - an function handle which returns a [nx2] (tall) np.array of box constraints
            constraints - a list of function handles to hard constraint functions (to be implemented)
            x0 - a guess for the location of xBest, used by some algorithms to initiate the optimization
        outputs:
            x - a list of np.array objects, each of which is a solution
                list of the parameter for the optimal solution
        """
        pass


<a id="AlgVis"></a>
# Optimization Algorithm Visualizer [ &#x21ea;](#OptTop)

* pick from a set of test functions, visualize how that optimizer works.

In [2]:
import plotly.graph_objs as go
import ipywidgets as widgets
from IPython.display import display
import numpy as np



class OptimAlgVis():
    """
    Optimization Algorithm Visualizer - test out the performance a sampling based
    optimization algorithm on a set of test objective functions
    """
    def __init__(self,opt,nSamps=1000):
        #-------------- test function and opt setup -------------
        self.opt = opt               # handle to the optimization algorithm object
        self.nSamps = nSamps         # how many samples to take
        self.data = []               # where you will store the samples leading upto the optimal point
        
        #import the test functions from landscapes library
        from landscapes.single_objective import  ackley
        from landscapes.single_objective import  eggholder
        from landscapes.single_objective import  himmelblau
        
        # list of test function handles to evaluate the optimizer on
        self.testFunctions = \
        [
            ackley,
            eggholder,
            himmelblau,
        ]
        
        
        #names of test functions
        self.tfNames = {"ackley":0,"eggholder":1,"himmelblau":2}
        
        #current test function (state)
        self.tf = None                     # init at call time of plot 
        self.tfid = None                   # id of the test function updated at call time
        
        #list of test fxn bounds
        self.testFunctionBounds = \
        [
            np.array([[-5,5],          # ackley
                      [-5,5]]),
            np.array([[-512,512],      # eggholder
                      [-512,512]]),
            np.array([[-5,5],          # himmelblau
                      [-5,5]]),
        ]
        
        #list of global optima for comparison
        self.globalOptima = \
        [
            [(0,0),0],                      # ackley
            [(512, 404.2319), -959.6407],   # eggholder
            [(3.0,2.0),0]                   # himmelblau (there are 4 global solutions, consider plotting all of them)
        ]
        
        #------------- visualization setup ---------------------
        self.figl = go.FigureWidget()
        self.figl.add_trace(go.Scatter3d(z=[0],x=[0],y=[0], name = "sampling",mode="markers",
                                                marker = dict(size=10,color = np.linspace(0,1,self.nSamps))))
        self.figr = go.FigureWidget()
        self.figr.add_trace(go.Scatter(x=[], y=[],mode = "markers"))
        
        s2s = widgets.IntSlider(min=1,max=self.nSamps,value=1,description='show')
        n   = widgets.IntSlider(min=1,max=self.nSamps,value=1,description='n')
        self.wdict = {"s2s":s2s,"n":n}
        
        self.figr.update_layout(
            #autosize=False,
            #width = 1400,
            height= 800,
            legend=dict(x=.025, y=.975),
            margin=dict(l=0, r=20, t=0, b=0))
        
        self.figr.update_layout(
            #autosize=False,
            #width = 1400,
            height= 800,
            legend=dict(x=.025, y=.975),
            margin=dict(l=100, r=100, t=200, b=150))
        
        
    def bounds(self):
        """
        a function which returns the correct bounds np.array object, given that the 
        internal state of which test function is active
        """
        return self.testFunctionBounds[self.tfid]
        
    def obj(self,x,grad=0):
        """
        objective function of the currently selected test function, which has 
        the side effect of writing [x, fx] into a databuffer for visualization
        """
        fx = self.tf(x)
        sln = list(x.copy())
        sln.append(fx)
        self.data.append(sln)
        return fx
        
    #----------------- visualization functions -------------------
    def plotTestFxn3D(self,fxnName):
        #extract function and assoicated objects
        fid = self.tfNames[fxnName]
        fxn = self.testFunctions[fid]
        b = self.testFunctionBounds[fid]
        
        # Generate data
        x = np.linspace(b[0,0],b[0,1] , 100)
        y = np.linspace(b[1,0],b[1,1] , 100)
        xGrid, yGrid = np.meshgrid(y, x)
        
        # define a function and vectorize it 
        # for simple application over a meshgrid
        @np.vectorize
        def vFunc(x,y,fxn):
            return fxn((x,y))
        
        #evaluate z
        z = vFunc(xGrid, yGrid,fxn)

        # adding surfaces to subplots.
        self.figl.add_trace(go.Surface(x=x, y=y, z=z, colorscale='Viridis', showscale=False))
        
    def getSamps(self,s2s,n):
        rng = range(n,n+s2s)
        xs = self.data[rng,0]
        ys = self.data[rng,1]
        zs = self.data[rng,2]
        return xs,ys,zs

    def update(self,s2s= 10,n=1):
        
        figl = self.figl
        with figl.batch_update():
            #get the sampled points
            xs,ys,zs = self.getSamps(s2s,n)
            
            figl.data[0]['x']= xs
            figl.data[0]['y']= ys
            figl.data[0]['z']= zs
            
        figr = self.figr
        with figr.batch_update():
            figr.data[0]['x']= xs
            figr.data[0]['y']= ys
            
            
    def disp(self):
        self.out = widgets.interactive_output(self.update, self.wdict)
        display(self.wdict["s2s"])
        display(self.wdict["n"])
        sideBySide = widgets.HBox([self.figl,self.figr])
        display(sideBySide)
 

    #------------------- interface ----------------
    def plot(self,fxnName = "ackley"):
        """
        call this function to test and plot the landscape and the sample points
        for the specified optmization algorithm
        input:
            fxn2plot, which of the test functions should be plotted?
        """
        # register the test function with self
        tfid = self.tfNames[fxnName]
        self.tf = self.testFunctions[tfid]
        self.tfid = tfid
        
        # setup the optimizer with the test function and perform optimization
        self.opt.nSamps = self.nSamps
        self.opt.optimize(self.obj,self.bounds)
        self.data = np.array(self.data)   #turn cached data into an array
        
        #---- setup the visualizer -------
        
        #plot the test fxn in 3D
        self.plotTestFxn3D(fxnName)
        
        #setup the bounds of the sampling plot
        b = self.testFunctionBounds[tfid]
        self.figr.update_xaxes(range=[b[0,0], b[0,1]])
        self.figr.update_yaxes(range=[b[1,0], b[1,1]])
        
        
        
        #plot the interactive visualization
        self.disp()
        
        
     

# sobol sampling


#### refs:
https://people.sc.fsu.edu/~jburkardt/py_src/sobol/sobol.html

In [156]:
from sobol import i4_sobol
import ipywidgets as widgets
from IPython.display import display
import plotly.graph_objs as go
import numpy as np

    
class Sobol:
    """
    the sobol opt class 
    """
    def __init__(self):
        self.n = 1000          #number of samples to evaluate
        self.nBest = 10        #number to keep
        self.xBest = []        #list of nBest x values
        self.fxBest = []       #list of nBest obj function values
        self.randomSeed = True #should sample seed be randomized?
        
        self.selectionFxn = "max_fx" #str name of selection fxn ["max_fx", "obj_var_tradeoff"]
        self.W_obj = 1                #importance parameter for objective fxn
        self.W_dispersion = 1         #importance parameter for dispersion metric 
        
        self.xs = []                  #n x's
        self.fxs = []                 #n solutions
        
    
    @staticmethod
    def twoPointLine(x,x1,y1,x2,y2):
        """returns y given the formula of a line specified in 2 point form"""
        return ((y2-y1)/(x2-x1))*(x - x1) + y1
    
    @staticmethod
    def Sample(bounds,n,randomSeed = True):
        """
        inputs:
            bounds - [nx2] array of [max,min]
            n - number of sample points requested
            randomSeed - bool - start in a place other than 0
        outputs:
            samps - list of np.arrays  (so you can interate through them more easily)
        """
        #calc the sobol inputs
        ndims = bounds.shape[0]
        upperbounds = bounds[:,0] ; lowerbounds = bounds[:,1]
        if randomSeed: seed_0 = np.random.randint(0,10000)
        else: seed_0 = 0
        
        #sample the unit hypercube
        samps = [i4_sobol(ndims, seed)[0] for seed in range(seed_0,n + seed_0)]
        #samps = np.array(samps)
        
        #shift and scale from the unit hypercube to the bounds rectangular prism
        sl = []  #samps list
        for sample in samps:
            sn = np.zeros(ndims)  #sample new
            for i,x in enumerate(sample):
                x1 = 0 ; x2 = 1
                y1 = lowerbounds[i] ; y2 = upperbounds[i]
                sn[i] = Sobol.twoPointLine(x,x1,y1,x2,y2)
            sl.append(sn)
        
        return sl
    
    
    #------------------ selection fxns ------------------------
    def max_fx(self):
        """
        return the nBest results based only on the value of Fx
        """
        pass
        
    def re_sample(self):
        pass
    
    def histSelect(self):
        """
        randomly select solutions from the top percent,and place them in the xBest
        and fxBest
        """
        nbest = 10
        ids = np.argsort(self.fxs)[::-1][:nbest]
        
        #place best solutions into list
        self.xBest  = [self.xs[i]  for i in ids]
        self.fxBest = [self.fxs[i] for i in ids]
    
    #------------------- optimization --------------------------
    def optimize(self,obj,bounds,constraints):
        """
       discover interesting and good solutions at different locations within the space, 
       no optimization is performed strictly speaking, just structured exploration
        """
        self.xs = Sobol.Sample(bounds,self.n,self.randomSeed)
        self.fxs = [] 
        for x in self.xs:
            self.fxs.append(obj(x))
        
        return eval('self.' + self.selectionFxn + '()')


class SobolVis:
    """
    just want to get a sense for what sampling a unit space
    using sobol sampling looks like
    """
    def __init__(self):
        _fig = go.Figure(data=[go.Scatter3d(z=[0],x=[0],y=[0], name = "sampling",mode="markers",
                                            marker = dict(size=10,color = np.linspace(0,1,1000),showscale=True))])
        self.fig = go.FigureWidget(_fig)
        n = widgets.IntSlider(min=1,max=1000,value=1,description='samps')
        self.wdict = {"n":n}
        
        self.fig.update_layout(
            autosize=False,
            width = 800,
            height= 800,
            legend=dict(x=.025, y=.975),
            margin=dict(l=0, r=20, t=0, b=0))
        
    def genSamps(self,n):
        #how to use sobol
        bounds = np.array([[.5,1,-1],[.25,-1,-3]]).T
        samps = Sobol.Sample(bounds,n,randomSeed = False)
        #samps = [i4_sobol(dim_num, seed)[0] for seed in range(n)]
        samps = np.array(samps)
        xs = samps[:,0] ; ys = samps[:,1] ; zs = samps[:,2]
        return xs,ys,zs
    
    
    def update(self,n=1):
        fig = self.fig
        with fig.batch_update():
            #plot n sobol points, with coloring
            xs,ys,zs = self.genSamps(n)
            
            fig.data[0]['x']= xs
            fig.data[0]['y']= ys
            fig.data[0]['z']= zs
            
            
    def disp(self):
        self.out = widgets.interactive_output(self.update, self.wdict)
        display(self.wdict["n"])
        display(self.fig)
        

#interactive 3D sobol sampling visualization to get a sense of the algorithm 
if not asLibrary() and True:
    vis = SobolVis()
    vis.disp()

IntSlider(value=1, description='samps', max=1000, min=1)

FigureWidget({
    'data': [{'marker': {'color': array([0.      , 0.001001, 0.002002, ..., 0.997998, 0.998999,…

### BFGS 
Broyden–Fletcher–Goldfarb–Shanno (BFGS) is a quazi-newton optimization method that uses the secant approach to find stationary points via locating the zeros of the gradient (jacobian) function. 

In [168]:
from scipy.optimize import minimize
from scipy.optimize import Bounds

class BFGS:
    def __init__(self):
        self.mode = "seeded"   #["seeded","sobol","SA"] 
        self.n = 1             #number of starting points
        self.x0 = []           #incase you want to prime the algorithm, from outside manipulation
        self.xBest = []        #list of n x values
        self.fxBest = []       #list of n obj function values
        self.res = []          #results of most recent optimization
        
    def optimize(self,obj,bounds,constraints): #refactor to remove this code repetition.
        
        #specify the bounds:
        bnds = Bounds(bounds[:,1],bounds[:,0],keep_feasible=True)
        
        #specify the fixed step size (epsilon):
        eps = (bounds[:,0] - bounds[:,1])/10000
        
        #make a maximum into a minimum
        min_obj = lambda x: -1 * obj(x)
        
        if self.mode == "seeded":
            self.res = minimize(min_obj,
                                self.x0[0],
                                method='BFGS',
                                jac = False,
                                bounds = bnds,
                                options={'gtol': 1e-3,
                                         'disp': True,
                                         'eps': eps})
                                        #'finite_diff_rel_step':finite_diff_rel_step})
            self.xBest.append(self.res.x)
            self.fxBest.append(1)
            
        elif self.mode == "sobol":
            samps = Sobol.Sample(bounds,self.n,randomSeed = True)
            for x0 in samps:
                self.res = minimize(obj, x0, method='BFGS',
                           options={'gtol': 1e-6, 'disp': True})
                self.xBest.append(self.res.x)
                self.fxBest.append(1)
        
        return self.xBest

<a id="bayes"></a>
# Baysian Optimization [ &#x21ea;](#OptTop)
- optimization based on the formation of a surogate model of a gaussian process


In [52]:
from skopt import Optimizer as _bayesOpt


class BayesOpt:
    """
    a baysian optimization algorithm based on the formation of a surogate
    model of a gaussian process
    """
    def __init__(self):
        self.nSamps = 20
        self.plotResults = False
        
    def optimize(self,obj,bnds,constraints=None,x0=None):
        
        #define boundary condition limits (lower,upper)
        bounds = bnds().astype(float)  #cast to a float so that optimizer doesn't think this is an integer problem.
        lb = bounds[:,0]
        ub = bounds[:,1]
        bounds = list(zip(lb,ub))

        #run the optimization
        opt = _bayesOpt(bounds, "GP", n_initial_points=10,acq_optimizer="sampling")
        for i in range(self.nSamps):
            suggested = opt.ask()
            y = obj(suggested)
            res = opt.tell(suggested, y)
        
        #plots (could expand on this)
        if self.plotResults:
            from skopt.plots import plot_objective, plot_evaluations
            plot_evaluations(res, bins=10)
            plot_objective(res)
        
#visualizer performance on test functions
if not asLibrary():
    names = {0:"ackley",1:"eggholder",2:"himmelblau"}
    alg = BayesOpt()
    vis = OptimAlgVis(alg,nSamps=20)
    vis.plot(names[0])
    

    



IntSlider(value=1, description='show', max=20, min=1)

IntSlider(value=1, description='n', max=20, min=1)

    'data': [{'marker': {'color': array([0.        , 0.05263158, 0.10526316, 0.1…

<a id="SA"></a>
# Simulated Annealing [ &#x21ea;](#OptTop)

In [11]:
#simulated anealing algorithm must have the following functions: 
# the feasable state space
# the Energetic function (which is the objective function) E()
# the candidate generator procedure (neighbor(x))
#        -generation of random candidate vector, with variable magnitude?
#         aniostropic exploration is better!!!
#thoughts for how I might solve this problem - 
#         boundry check or only sampling within the boundry seems a good strategy. 
#        -moving in one direction at at time. 

# acceptance probability function P()
# anealing schedule - number, number of anealing cycles, cooling rate
# some termination condition - total execution time or number of interations
# presensce of and number of restarts, deterministic or otherwise? 

#of these, cooling schedule and neighbor seem the most important functions


class SA:
    def __init__(self):
        self.nSamps = 1000;                    #total number of iterations to perform
        self.nResets = 3                       #number of resets to best 
        self.brt = self._brt()                 #trial numbers during which to reset to best
        self.dist = "uniform"                  #neighborhood sampling method 
        self.aMode = "Exp"                     #Annealing mode ["exp","linear","stepped"]
        self.nMode = "Linear"                  #neighborhood Mode
        self.T = 1                             #temperature variable on [1-0)
        self.N = 1                             #neighborhood size coefficient [1-0)
        self.C_half = 200                      #half-life cooling              [trials]
        self.N_half = 150                      #half-life of neighborhood size [trials]
        self.fxBest = 0                        #best encountered function value
        self.xBest = []                        #location of best value
        self.dbg = False                        #save variables for debugging
        self.xCache  = []                      #storage for x
        self.FxCache = []                      #storage for Fx
        self.updateReports = True              #should we print out an update of how the optimization is progressing?
        self.updateCount = 10                  #number of updates
        
        
    def setBounds(self,bcs):
        self.x_min = bcs[:,0]                  #upper limits on box constraints for x variable
        self.x_max = bcs[:,1]                  #lower limits on box constraints for x variable
        
    def _center(self,x_max,x_min):
        """
        find  the center of a box defined by limits
        """
        return [(x_max[i] + x_min[i])/2 for i in range(len(x_max))]
    
    def _twoPointLine(self,x,x1,y1,x2,y2):
        """
        returns y given the formula of a line specified in 2 point form
        """
        return ((y2-y1)/(x2-x1))*(x - x1) + y1
    
    def _brt(self):
        """
        calculate the best reset trials, evenly distribute in n_iter
        """
        return np.random.randint(low  = int(self.nSamps/5),
                                 high = self.nSamps,
                                 size = self.nResets)
        
            
    def sample(self,x):
        """
        sample the space for the next sample based on the neighborhood
        function
        input:
            x
        output:
            x_new
        """
        #define the (ever-shrinking) neighborhood bounding box
        w = self.N * ((self.x_max - self.x_min)) 
        nx_max = x + w
        nx_min = x - w
        
        #define the combined bounding box as an intersection between
        #the box constraints and the neighborhood box - a mini-max problem
        cx_max = np.amin(np.vstack((self.x_max,nx_max)),axis = 0)
        cx_min = np.amax(np.vstack((self.x_min,nx_min)),axis = 0)
        
#         for i in range(len(self.x_max)):
#             cx_max[i] = min(self.x_max,nx_max[i])
#             cx_min[i] = max(self.x_min,nx_min[i])
        
        #sample from the combined box, according to the appropriate distribution
        if self.dist == "uniform": 
            x_new = np.random.uniform(cx_min,cx_max) #vectorized
            
        if self.dist == "normal":
            pass
#             zScore = 1.645 #95% confidence interval
#             x_new = []
#             while len(x_new) > len(self.x_min):
#                 x = np.random.normal(loc=0.0, scale=1.0, size=None)
#                 if abs(x) < zScore:
#                     x_new.append(x)
#             c_range = cx_max - cx_min
            
 
        return x_new

    def optimize(self,obj,bounds,constraints=None,x0=None):
        
        #set box constraints
        bcs = bounds()
        self.setBounds(bcs)
        
        #init x0 and fx0
        x0 =  self._center(self.x_max,self.x_min)
        fx0 = obj(x0)
        
        for n in range(self.nSamps + 1):
            x = self.sample(x0)
            fx = obj(x)
            
            #if better, always keep
            if fx > fx0:
                x0 = x ; fx0 = fx
                
            #probabalistically accept worse answer
            else:
                Δf = fx - fx0
                r = np.random.random()
                if r < np.exp(Δf/self.T):
                    fx0 = fx ; x0 = x
                    #print(x)
                else:
                    pass
             
            
            #maintain best (for restarts, if used)
            if fx > self.fxBest:
                self.xBest = x  ; self.fxBest = fx 
                
            #do a best restart:
            if n in self.brt:
                  x0 = self.xBest ; fx0 = self.fxBest 
            
            #decrease temperature and neighborhood size
            if self.aMode == "Exp": self.T = .5**(n/self.C_half)
            if self.nMode == "Exp": self.N = .5**(n/self.N_half)
            
            if self.aMode == "Linear": self.T = self._twoPointLine(n,0,1,self.nSamps,0)
            if self.nMode == "Linear": self.N = self._twoPointLine(n,0,1,self.nSamps,0)
                  
            #store debugging vars
            if self.dbg:
                self.xCache.append(x0[0])
                self.FxCache.append(fx0[0])
                
            #give updates
            if self.updateReports:
                if n % ((self.nSamps) / 10) == 0:
                    print("on iteration %d" %n)
            
        return x0
                    
 



#visualizer performance on test functions
if not asLibrary():
    names = {0:"ackley",1:"eggholder",2:"himmelblau"}
    alg = SA()
    vis = OptimAlgVis(alg)
    vis.plot(names[2])
    

on iteration 0
on iteration 100
on iteration 200
on iteration 300
on iteration 400
on iteration 500
on iteration 600
on iteration 700
on iteration 800
on iteration 900
on iteration 1000


IntSlider(value=1, description='show', max=1000, min=1)

IntSlider(value=1, description='n', max=1000, min=1)

    'data': [{'marker': {'color': array([0.      , 0.001001, 0.002002, ..., 0.99…

<a id="DIRECT"></a>
# DIRECT Algorithm [ &#x21ea;](#OptTop)

the direct algorithm is a global optimization method that breaks up a bounded hyper-rectangle search space into progressively smaller and smaller sub-rectangles by identifying (via checking the center point of the rectangle) which areas of the input space are most prominsing. 



In [12]:
import nlopt

class NLopt:
    """
    container class which set's up the default configuration for a 
    NLopt algorithm.
    """
    def __init__(self,method = "DIRECT_L"):
        
        #simulation parameters which can be configured during experiment
        self.nSamps = 10000;                                                 # total number of samples to take
        if method == "DIRECT_L": self.algorithm = "GN_DIRECT_L"              # default NLopt algorithm direct w/ local bias
        elif method == "DIRECT": self.algorithm = "GN_DIRECT"                # DIRECT algorithm
        elif method == "CRS"   : self.algorithm = 'GN_CRS2_LM'               # CRS
        elif method == "ES"    : self.algorithm = 'GN_ISRES'                 # evolutionary strategy
        elif method == "EA"    : self.algorithm = 'GN_ESCH'                  # evolutionary algorithm
        
        
    def optimize(self,obj,bnds,constraints=None,x0=None):
        
        #init the optimization object
        bounds = bnds()                             # get bounds array
        n = bounds.shape[1]                         # number of parameters
        
        #setup optimizer interface
        alg = eval("nlopt." + self.algorithm)
        opt = nlopt.opt(alg, n)

        #set bound constraints
        opt.set_lower_bounds(bounds[:,0])
        opt.set_upper_bounds(bounds[:,1])

        #set the stop value
        opt.set_maxeval(self.nSamps)

        #register the objective function
        #opt.set_max_objective(obj)
        opt.set_min_objective(obj)

        #execute optimization
        if not x0: x0 = [np.random.uniform(bounds[i,0],bounds[i,1]) for i in range(n)]
        xopt = opt.optimize(x0)
        
        #return optimization results
        result = opt.last_optimize_result()
        return xopt


        
#visualizer performance on test functions
if not asLibrary():
    names = {0:"ackley",1:"eggholder",2:"himmelblau"}
    alg = NLopt()
    vis = OptimAlgVis(alg)
    vis.plot(names[2])
    


IntSlider(value=1, description='show', max=1000, min=1)

IntSlider(value=1, description='n', max=1000, min=1)

    'data': [{'marker': {'color': array([0.      , 0.001001, 0.002002, ..., 0.99…

<a id="CMA"></a>
# Covariance Matrix Adaptation (CMA) [ &#x21ea;](#OptTop)

* description

In [61]:
import cma

# notes on CMA
# there appears to be a need to scale the problem (by the bounds) such that the solution is within 3 sigma,
# and one value of sigma is sufficient for each variable, meaning that x should be scaled?? 
# use CMAevolutionaryStrategy interface. - loop control stays with user
# reading indicates it might be a good idea to repeat an optimization multiple times with an increaseing population schedule?
# there seems to be a bit of an art to using this algorithm... return to this at a later date perhaps?


# this is still a work in progress, as boundries are not well implemented, however for now we can ICE this, as we have more than enough other implementations



class CMA: 
    def __init__(self):
        self.popSize = 100
        self.generations = 10 
        self.nSamps = self.popSize * self.generations
        self.mcPrime = True       # MonteCarlo Prime for x0?
        self.mcSamps = 100        # MonteCarlo Samples
        self.σ = 2                # 1/5th of solution space
        self.lb = None            # init this when optimize gets called
        self.ub = None            # init this when optimize gets called
        
    def scale(self,x):
        """
        perform a linear scaling from a point in the problem domain
        to a point in a scaled domain, xscl
        inputs: 
            x - a point in the problem domain in Rn
        outputs: 
            xscl - a point defined on [0-10] over n dimensions
        """
        return self.lb + (self.ub-self.lb) * x / 10  # vectorized expression w/ elementwise operations
            
        
    def scaleInv(self,xscl):
        """
        perform an inverse operation to self.scale, moving from the 
        scaled domain to the original problem domain
        inputs: 
            xscl - a point defined on [0-10] over n dimensions
        outputs:
            x - a point in the problem domain in Rn
        """
        return (xscl - self.lb)*10 / (self.ub-self.lb) # vectorized expression w/ elementwise operations
        
    
    def objWrap(self,obj):
        """
        wrap the objective function in an invScale function
        so that it's dealing with vlues in the proper domain
        """
        def wrap(x,grad=None):
            x = self.scaleInv(x)
            return obj(x)
        return wrap
        
    
    def optimize(self,obj,bnds,constraints=None,x0 = None):
        
        #prime with Monte-carlo samples to get x0
        if self.mcPrime == True and x0 == None:
            primeAlg = NLopt()
            primeAlg.nSamps = self.mcSamps
            x0 = primeAlg.optimize(obj,bnds)
            print(x0)
        
        #setup the lower and upper bounds in internal object
        bounds = bnds()
        self.lb = bounds[:,0]
        self.ub = bounds[:,1]
        
        
        #setup the bounds of the algorithm in the scaled domain
        opts = {'bounds':[0,10]}
        
        
        #wrap the objective function so that the assumption that every var is on [0-10] is valid
        obj_w = self.objWrap(obj)
        
        #set other termination criteria
        
        #run the optimization
        es = cma.CMAEvolutionStrategy(x0, self.σ,opts)
        i = 0
        while not es.stop() and i < self.generations:
            solutions = es.ask()
            es.tell(solutions, [obj_w(s) for s in solutions])
            #es.disp()
            i += 1 
            #es.result_pretty()
        return self.scale(es.result[0])
            

        
#visualizer performance on test functions
if not asLibrary():
    names = {0:"ackley",1:"eggholder",2:"himmelblau"}
    alg = CMA()
    vis = OptimAlgVis(alg)
    vis.plot(names[2])
    
        
        

[4.81481481 4.81481481]
(3_w,6)-aCMA-ES (mu_w=2.0,w_1=63%) in dimension 2 (seed=367358, Mon Mar  8 21:00:03 2021)


IntSlider(value=1, description='show', max=1000, min=1)

IntSlider(value=1, description='n', max=1000, min=1)

    'data': [{'marker': {'color': array([0.      , 0.001001, 0.002002, ..., 0.99…

In [62]:
# import cma
# #help(cma)  # "this" help message, use cma? in ipython
# #help(cma.fmin)
# help(cma.CMAEvolutionStrategy)

# cma.CMAOptions('tol')  # display 'tolerance' termination options
# cma.CMAOptions('verb') # display verbosity options
# res = cma.fmin(cma.ff.tablet, 15 * [1], 1)
# es = cma.CMAEvolutionStrategy(15 * [1], 1).optimize(cma.ff.tablet)
# help(es.result)
# res[0], es.result[0]  # best evaluated solution
# res[5], es.result[5]  # mean solution, presumably better with noise


<a id="Notes"></a>
# Notes: [ &#x21ea;](#OptTop)


##  optimizer revamp plan of action

### to do's: 


* finish CMA class with scale wrapper and connect to visualizer
* select 2- 3  more optimizers from NLopt, and make a way to easily change between them, with iterations as the termination criteria.
* ~Strike~
* ~finish the outline of the optimizers library~
* ~build the DIRECT class~
* finish up some CMA notes
* implement CMA to the point where you can run the 2D test functions, but don't implement the objective function rescaleing yet, you can do that at some future point
* finish the test function visualizer, and then visualize the three optimization algorithms
* change the way that torque is parameterized. this will take a bit of thinking, but I believe the best way to do it is to have a function that trades off torque between the right and left motors. the reason to do this is that if you allow 2 prameters for torque, then it will always be beneficial to add torque. we proposed normalization as a way to address this, and that still might be reasonable, but if we dont have normalization, perhaps we could have a torque budget(200nm) and then the algorithm could trade off additively. between the two torques. x + y = 100 we could then titrate this in, to see where we stop getting additional value from adding more system torque capability, and also if the ratios stay similar (i feel like they should, but who knows?)
* integrate optimizers back into experiment, and start setting up some experiments in ExpsForPaper, including torque experiments. 
* once you have a feel for how these exps are turning out, it's time to plan out the figs for the paper, and the story for the prelim.



### plan:

* leave BDGS and Sobol here for now, there isn't a point in getting rid of them yet, and they might be useful.
* end objective: 3 optimization classes, and a visualizer class that takes a handle to an optimizer, and a 2D example function, and a number of iterations, and plots points over time. 
* total time - 4 - 6 hours
* visualizer - 2.5 hrs
* DIRECT and CMA classes - 2hrs
* 1hr to tie it all together 
* 10min clean up
